In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv('D:\\Datasets\\Winsconsin\\data.csv')

In [3]:
raw_data = raw_data.drop(columns=[raw_data.columns[-1],raw_data.columns[0]],axis=1)

In [4]:
raw_data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
#split data according to class
m_data = raw_data[raw_data['diagnosis']=='M']
b_data = raw_data[raw_data['diagnosis']=='B']

In [6]:
#evenly split train data
train = pd.concat([m_data.iloc[0:170],b_data.iloc[0:170]])
test = pd.concat([m_data.iloc[170:],b_data.iloc[170:]])

In [7]:
train['diagnosis'].replace(to_replace=['B','M'],value=[0,1],inplace=True)
labels = train['diagnosis']
labels = np.array(labels)
labels = labels.reshape(train.shape[0],1)

In [9]:
#Z-score normalisation
training_data = (train.iloc[:,1:])

mu=[]
std=[]
for i in range(training_data.shape[1]):
    mu.append(training_data.iloc[:,i].mean())
    std.append(training_data.iloc[:,i].std())

In [11]:
def Decision_boundary(data,params):
    '''
    this function calculates the decision boundary
    y = theta*X + thet0
    '''
    ans = np.matmul(data,params[0]) + params[1]
    #print('ans',ans.shape)
    return ans

In [12]:
def Sigmoid(plane):
    return np.array((1/(1+np.exp(-plane))))

In [13]:
def Neg_log_loss(H,labels,thet):
    a = (-1/labels.shape[0])*(np.sum(np.matmul(np.transpose(labels),np.log(H)))\
                               + np.sum(np.matmul(np.transpose(1-labels),np.log(1-H))))
    return a

In [14]:
def fdash_thet0(H,labels):
    a=np.sum(labels - H)
    return np.sum(labels - H)

In [15]:
def fdash_thet(H,labels):
    score = np.asarray(H,dtype=float)
    labels = np.asarray(labels,dtype=float)
    ans = H - labels
    ans = np.matmul(np.transpose(ans),training_data) + (0.01/340)*np.sum(H)
    return np.transpose(ans)

In [16]:
#initialize thetas for gradient descent
training_data=np.array(training_data_1)
thet0_i = np.zeros(shape=(1,1))

thet_i = np.zeros(shape=(30,1))

epsilon = 10e-7

learning_rate = 10e-11

iterations = []

neg_log_history = []
score_vals = []

i=0

while True:
    plane_i = Decision_boundary(training_data,[thet_i,thet0_i])
    H_i = Sigmoid(plane_i)
    
    thet0_f = thet0_i - learning_rate*(fdash_thet0(H_i,labels))
    thet_f = thet_i - learning_rate*(fdash_thet(H_i,labels))
    
    plane_f = Decision_boundary(training_data,[thet_f,thet0_f])
    score_vals.append(plane_f)
    H_f = Sigmoid(plane_f)
    
    neg_loss = Neg_log_loss(H_i,labels,thet_i) - Neg_log_loss(H_f,labels,thet_f)
    neg_log_history.append(Neg_log_loss(H_i,labels,thet_i))
    iterations.append(i)
    
    if i%5000==0:
        print('neg log loss and loss at iteration {} is {} {}'.format(i,neg_loss,Neg_log_loss(H_f,labels,thet_f)))
    
    i+=1
    
    if abs(neg_loss)<epsilon:
        print('loss at iteration {} is {}'.format(i,Neg_log_loss(H_f,labels,thet_f)))
        break
    else:
        thet0_i = thet0_f
        thet_i = thet_f  

neg log loss and loss at iteration 0 is 0.0020890626938873114 0.691058117866058
neg log loss and loss at iteration 5000 is 9.799334434790197e-06 0.563017310675908
neg log loss and loss at iteration 10000 is 6.773230262102636e-06 0.5224413687295089
neg log loss and loss at iteration 15000 is 5.1143504749551205e-06 0.49308242033194444
neg log loss and loss at iteration 20000 is 4.079283290381319e-06 0.4702803694192343
neg log loss and loss at iteration 25000 is 3.376720672298994e-06 0.4517439154845446
neg log loss and loss at iteration 30000 is 2.870588254877404e-06 0.4361896686982297
neg log loss and loss at iteration 35000 is 2.489379972603345e-06 0.4228318586537948
neg log loss and loss at iteration 40000 is 2.192152454061347e-06 0.4111570802104889
neg log loss and loss at iteration 45000 is 1.9538847831901407e-06 0.4008128114704929
neg log loss and loss at iteration 50000 is 1.7584875328102711e-06 0.3915472837315647
neg log loss and loss at iteration 55000 is 1.5951830318883253e-06 0

In [17]:
#training accuracy
train_vals=[]
for i in Sigmoid(score_vals[-1]):
    if i<0.5:
        train_vals.append(0)
    else:
        train_vals.append(1)
train_vals = np.array(train_vals)
train_vals = train_vals.reshape(train.shape[0],1)

In [18]:
(np.count_nonzero(train_vals==labels)/train.shape[0])

0.9088235294117647

In [19]:
#prep test data
test.replace(to_replace=['B','M'],value=[0,1],inplace=True)

In [21]:
test_labels = test['diagnosis']
test_labels = np.array(test_labels).reshape(test.shape[0],1)

In [33]:
#testing accuracy
test_scores = Decision_boundary(np.array(test.iloc[:,1:]),[thet_f,thet0_f])

test_vals=[]

for i in Sigmoid(test_scores):
    if i<0.5:
        test_vals.append(0)
    else:
        test_vals.append(1)
test_vals = np.array(test_vals)
test_vals = test_vals.reshape(test.shape[0],1)
(np.count_nonzero(test_vals==test_labels)/test.shape[0])

0.9170305676855895